# Simulator

`amaranth.sim`模块，也被称为模拟器，使得在硬件实现之前呢狗狗在虚拟环境中评估设计功能。

## 模拟电路

以下示例模拟了两个设计中的一个：在同步始终域中运行的同步计数器和组合加法。

In [21]:
%config InteractiveShell.ast_node_interactivity = "all"

from amaranth.lib import wiring
from amaranth.lib.wiring import In, Out

class Counter(wiring.Component):
    en: In(1, init=1)
    count: Out(4)

    def elaborate(self, platform):
        m = Module()
        with m.If(self.en):
            m.d.sync += self.count.eq(self.count + 1)
        return m

class Adder(wiring.Component):
    a: In(16)
    b: In(16)
    o: Out(17)

    def elaborate(self, platform):
        m = Module()
        m.d.comb += self.o.eq(self.a + self.b)
        return m

## 测试模拟电路

模拟一般需要三个基本步骤：构建被测设备（DUT），为其构建一个模拟器，以及使用`Simulator.run()`或`Simulator.run_until()`方法运行模拟：

In [24]:
from amaranth.sim import Simulator, Period

dut = Counter()
sim = Simulator(dut)
sim.run()

ImportError: cannot import name 'Period' from 'amaranth.sim' (/home/weig/.local/lib/python3.10/site-packages/amaranth/sim/__init__.py)

然而，上面的代码没有激励，也没有测量被测设备的输出。

如果没有激励被添加到模拟中，Simulator.run()方法会立即返回。需要进行修改：

* `Simulator.add_clock()`方法添加激励
* `Simulator.run_until()`方法会在指定数量的时钟周期停止。
* `Simulator.write_vcd()`方法输出VCD格式的波形文件

In [25]:
dut = Counter()
sim = Simulator(dut)
sim.add_clock(Period(MHz=1)) # 1MHZ period 1us
with sim.write_vcd("example1.vcd"):
    sim.run_until(Period(MHz=1) * 15) # 15 periods of clock

NameError: name 'Module' is not defined

波形被保存到example1.vcd文件中，使用Sufer或GTKWave进行查看

`Simulator.reset()`方法将仿真恢复到初始状态。他可以用于加速测试，只是仿真被确定会遇到错误时输出波形：

In [5]:
try:
    sim.run()
except:
    sim.reset()
    with sim.write_vcd("example1_err.vcd"):
        sim.run

NameError: name 'sim' is not defined

## 测试同步电路

为了验证北侧设备在仿真中的功能是否正常，会提供已知值作为输入，并将输出与预期结果进行比较。

这是通过向仿真其添加一种不同类型的激励，一个testbench： 使用的是python `async`异步实现的。它被测试单元（DUT）并发运行，并可以操控模拟中使用的信号。测试平台通过`Simulator.addtestbench（）`方法添加，并通过`SimulatorContext`对象与模拟器近些年个交互：使用`ctx.get()`方法检查信号的value，使用`ctx.set()`方法更改信号的Value，或者使用`ctx.tick()`方法等待一个时钟的上升沿。

以下示例模拟了一个计数器，并验证了他可以通过`en`输入停止

In [26]:
dut = Counter()

async def testbench_example2(ctx):
    await ctx.tick().repeat(5)     # 等待第五个上升沿之后
    assert ctx.get(dut.count) == 5 # 验证counter是否为期待值
    ctx.set(dut.en, False)         # 关闭计数器
    await ctx.tick().repeat(5)     # 等待第10个上升沿之后
    assert ctx.get(dut.count) == 5 # 验证counter是否还是5
    ctx.set(dut.en, True)          # 打开计数器

sim = Simulator(dut)
sim.add_clock(Period(MHz=1))
sim.add_testbench(testbench_examole2) # 添加testbench
with sim.write_vcd("example2.vcd"):
    sim.run_until(Period(MHz=1) * 15)

NameError: name 'Module' is not defined

由于该电路是同步的，并且`ctx.tick()`方法在电路对上升沿反应后才返回，因此对`en`输入的改变会影响在改变后的下一个时钟周期电路行为

## 测试组合电路
在测试组合电路时，使用`ctx.delay()`方法来推进仿真时间，而不是`ctx.tick()`方法，因为在这种情况下仿真不包含时钟。`Simulator.run()`方法在所有测试平台执行完成后停止仿真并返回。

In [7]:
dut = Adder()

async def testbench_example3(ctx):
    await ctx.delay(Period(us=1))
    ctx.set(dut.a, 2)
    ctx.set(dut.b, 2)
    assert ctx.get(dut.o) == 4

    await ctx.delay(Period(us=1))
    ctx.set(dut.a, 1717)
    ctx.set(dut.b, 420)
    assert ctx.get(dut.o) == 2137

    await ctx.delay(Period(us=2))

sim = Simulator(dut)
sim.add_testbench(testbench_example3)
with sim.write_vcd("example3.vcd"):
    sim.run()

NameError: name 'Adder' is not defined

由于这个电路完全是组合逻辑电路，而且Amaranth模拟器使用的是组合电路零延迟模型，因此输出在输入变化的统一瞬间发生变化。

## 用代码代替电路
在仿真过程中，可以用等效的Python代码代替Amaranth电路。这可以用来提高仿真性能或避免在不需要合成的情况下将复杂的Python算法重新实现为Amaranth。

这是通过向模拟器添加一个进程来完成的：一个异步的Python函数，他与北侧设备（DUT）同时作为模拟的一个组成部分运行。
通过`Simulator.add_process()`方法添加一个进程，并接受一个SimulatorContex对象，通过该对象可以与模拟器进行交互。进程在概念上类似与testbench，但在两个重要方面有所不同。

* testbench一明确的顺序运行（从地一个到最后一个，按照添加的顺序，仅在等待点提供控制），并且不能观察到设计不一致的中间状态，但在输入发生变化后，processes以不确定的顺序运行，直到设计收敛。
* 在进程中，无法使用`ctx.get()`方法检查信号的Value，这确保了设计的中间不一致状态不会被整个过程观察到。

线程通过信号与设计的其余部分进行通信，就像一个可详细说明的东西一样。

## 替换同步电路
进程无法使用`ctx.get()`方法检查信号的Value。相反，在同步进程中，信号的值在每个时钟上升沿使用`ctx.tick()`方法进行采样。

以下代码将在一个进程中等效的Python代码替换Counter可拓展部分，并使用测试平台验证其正确性。

In [8]:
m = Module()
m.domains.sync = cd_sync = ClockDomain()
en = Signal(init=1)
count = Signal(4)

async def process_example4(ctx):
    count_value = 0
    async for clk_edge, rst_value, en_value in ctx.tick().sample(en):
        if rst_value:
            count_value = 0
        elif clk_edge and en_value:
            count_value += 1
            ctx.set(count, count_value)

async def testbench_example4(ctx):
    await ctx.tick().repeat(5)
    assert ctx.get(count) == 5
    ctx.set(en, False)
    await ctx.tick().releat(5)
    assert ctx.get(count) == 5
    ctx.set(en, True)

sim = Simulator(m)
sim.add_clock(Period(MHz=1))
sim.add_process(process_example4)
sim.add_testbench(testbench_example4)
with sim.write_vcd("example4.vcd", traces=(cd_sync.clk, cd_sync.rst, en, count)):
    sim.run()

NameError: name 'Module' is not defined

除非另有指示，否则`Simulator.write_vcd()`方法仅捕获在模拟器创建时提供给模拟器的电路中出现的信号。`en`和`count`信号不在其中，因此使用traces参数显式的添加他们，以便他们出现在VCD文件中。

## 替换组合电路
在组合电路中，通常使用`ctx.changed()`进行采样。
以下代码将可展开的加法器替换为等效的Python进程代码。

In [9]:
m = Module()
a = Signal(16)
b = Signal(16)
o = Signal(17)

async def process_example5(ctx):
    async for a_value, b_value, in ctx.changed(a, b):
        ctx.set(o, a_value + b_value)

async def testbench_example5(ctx):
    await ctx.delay(Period(us=1))
    ctx.set(a, 2)
    ctx.set(b, 2)
    assert ctx.get(o) == 4

    await ctx.delay(Period(us=1))
    ctx.set(a, 1717)
    ctx.set(b, 420)
    assert ctx.get(o) == 2137

    await ctx.delay(Period(us=2))

sim = Simulator(m)
sim.add_process(process_example5)
sim.add_testbench(testbench_example5)
with sim.write_vcd("exmaple5.vcd", traces=[a, b, o]):
    sim.run()

NameError: name 'Module' is not defined